In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기

In [2]:
##### 파일 불러오기 #####
df = pd.read_csv("C:/Users/user/Desktop/2021 빅콘테스트_데이터분석분야_퓨처스리그_ECO제주\data_processing/2_3_short_term_frgn_3_resident_pop_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'visit_pop_cnt', 'total_pop']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,visit_pop_cnt,total_pop
base_date,,,,
2018-01-31,2018-01-31,50110250,58090.5960,24419
2018-02-28,2018-02-28,50110250,69710.8394,24427
2018-03-31,2018-03-31,50110250,98026.6381,24532
2018-04-30,2018-04-30,50110250,103289.7891,24606
2018-05-31,2018-05-31,50110250,115255.9750,24715
...,...,...,...,...
2021-02-28,2021-02-28,50130620,280.6159,4173
2021-03-31,2021-03-31,50130620,1475.8404,4178
2021-04-30,2021-04-30,50130620,1283.0997,4161


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'visit_pop_cnt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,total_pop
base_date,,
2018-01-31,50110250,24419
2018-02-28,50110250,24427
2018-03-31,50110250,24532
2018-04-30,50110250,24606
2018-05-31,50110250,24715
...,...,...
2021-02-28,50130620,4173
2021-03-31,50130620,4178
2021-04-30,50130620,4161


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130620 ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,total_pop
base_date,
2018-01-31,4076
2018-02-28,4096
2018-03-31,4123
2018-04-30,4167
2018-05-31,4193
2018-06-30,4234
2018-07-31,4254
2018-08-31,4302
2018-09-30,4333


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'total_pop': base_date
 2018-01-31    4076
 2018-02-28    4096
 2018-03-31    4123
 2018-04-30    4167
 2018-05-31    4193
 2018-06-30    4234
 2018-07-31    4254
 2018-08-31    4302
 2018-09-30    4333
 2018-10-31    4368
 2018-11-30    4397
 2018-12-31    4414
 2019-01-31    4421
 2019-02-28    4401
 2019-03-31    4406
 2019-04-30    4382
 2019-05-31    4364
 2019-06-30    4342
 2019-07-31    4352
 2019-08-31    4331
 2019-09-30    4324
 2019-10-31    4295
 2019-11-30    4280
 2019-12-31    4247
 2020-01-31    4260
 2020-02-29    4259
 2020-03-31    4235
 2020-04-30    4223
 2020-05-31    4197
 2020-06-30    4208
 2020-07-31    4212
 2020-08-31    4203
 2020-09-30    4199
 2020-10-31    4197
 2020-11-30    4201
 2020-12-31    4200
 2021-01-31    4192
 2021-02-28    4173
 2021-03-31    4178
 2021-04-30    4161
 2021-05-31    4178
 2021-06-30    4152
 Name: total_pop, dtype: int64}

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)
#sub_df_float.tail()

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -2.210701
p-value : 0.202384
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 이 파일 안되고 log파일로 넘어가세요 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -3.256938
p-value : 0.016917
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
#!pip install pmdarima

In [10]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

##### 여기서 아래쪽에 'Best model:  ARIMA(0,1,0)(0,0,0)[1] intercept' 이렇게 나오면 (0,1,0) 가져다 쓸거임 #####
##### ARIMA(0,1,0) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=502.222, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=461.019, Time=0.06 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=427.526, Time=0.25 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=386.556, Time=0.10 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=382.431, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.28 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=437.967, Time=0.20 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=368.491, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=373.808, Time=0.12 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=373.951, Time=0.19 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=361.066, Time=0.16 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=376.544, Time=0.25 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept   : AIC

ARIMA(order=(3, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [11]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.total_pop.values, order=(3,0,0)) ##### 여기 order=(0,1,0)에 위에서 나온 숫자 넣는거 #####
                                    ##### 그리고 sub_df.card_amt.values에서 가운데 card_amt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 0)   Log Likelihood                -173.955
Method:                       css-mle   S.D. of innovations             14.292
Date:                Sat, 11 Sep 2021   AIC                            357.911
Time:                        01:31:09   BIC                            366.599
Sample:                             0   HQIC                           361.096
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4224.1351     45.480     92.879      0.000    4134.996    4313.275
ar.L1.y        1.1617      0.122      9.542      0.000       0.923       1.400
ar.L2.y        0.4134      0.223      1.857      0.0

# 예측하기

In [12]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값임 #####
##### 7월 : 10681271.81014634, 8월 : 10741037.02029268 #####
##### 이 값 가져다 엑셀파일에 입력하기 #####

(array([4160.8598085 , 4149.74108179]), array([14.29181106, 21.9066631 ]), array([[4132.84837355, 4188.87124346],
       [4106.80481108, 4192.67735249]]))
